<a href="https://www.kaggle.com/code/obinna11/titanic-machine-learning?scriptVersionId=126463726" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
import re
import lightgbm as lgbm

In [2]:
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")
PassengerId = test["PassengerId"]

y = train["Survived"]
X = train.drop("Survived" , axis = 1)

In [3]:
print(train.shape)
print(train.head())
print(train.isnull().sum())

(891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN    

In [4]:
#feature eng
all_dataset = pd.concat([X, test])
all_dataset["title"] = all_dataset["Name"].apply(lambda row: re.split(", |. |\*", row)[1])
all_dataset["title"] = all_dataset["title"].apply(lambda row: "other" if all_dataset["title"].value_counts()[row]<8 or row == "" else row)

In [5]:
#selecting the needed features
drop_columns = ['Name','PassengerId', 'Cabin','Ticket']
all_dataset.drop(drop_columns, axis =1, inplace= True)

all_dataset['Embarked'].fillna(all_dataset['Embarked'].mode()[0],inplace=True)
categorical_var = ['Sex', 'Embarked', 'title']
ordinal_encoder = OrdinalEncoder()
all_dataset[categorical_var] = ordinal_encoder.fit_transform(all_dataset[categorical_var])

In [6]:
#onehot encoding
one_hot = ['Pclass', 'Sex', 'Embarked', 'title']
OH_encoder = OneHotEncoder(handle_unknown = "ignore", sparse = False)
OH_dataset = pd.DataFrame(OH_encoder.fit_transform(all_dataset[one_hot]))
OH_dataset.index = all_dataset.index
all_dataset.drop(one_hot, axis =1, inplace =True)
all_dataset = pd.concat([all_dataset, OH_dataset], axis =1)

In [7]:
#imputing missing values/ age and fare
knn_imp = KNNImputer(n_neighbors=5)
all_dataset.loc[:, :] = knn_imp.fit_transform(all_dataset)
print(all_dataset.isnull().sum())

Age      0
SibSp    0
Parch    0
Fare     0
0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
dtype: int64


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [8]:
#split dataset back into train and test
X = all_dataset.iloc[:891,:]
test = all_dataset.iloc[891:,:]

In [9]:
#GradientBoosting training and prediction
rf = GradientBoostingClassifier(n_estimators=10)
rf.fit(X,y)
result = rf.predict(test)

output = pd.DataFrame({'PassengerId': PassengerId, 'Survived': result})
output.to_csv('submission.csv', index=False)

#LGBMClassifier training and prediction
rf = lgbm.LGBMClassifier(n_estimators=10)
rf.fit(X,y)
result = rf.predict(test)

output = pd.DataFrame({'PassengerId': PassengerId, 'Survived': result})
output.to_csv('submission2.csv', index=False)

#Only just a marginal imporvement in results between GradientBoosting and LGBMClassifier

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
